## Data discovery: Load and query Yellow Taxi data
> Download the dataset from [the official TLC Trip Record Data website](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page)

---

### This cell only shows how to document code
```python
# Load file
local_file = 'datasets/your-downloaded-from-TLC-taxis-file-here.parquet'

# Show data
spark.read.parquet(local_file).show()
```

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col

### What is master(local N)?
The --master option specifies the master URL for a distributed cluster, or local to run locally with one thread, or local[N] to run locally with N threads.

<b>Source</b>: See Spark [docs here](spark.apache.org/docs/latest). See all [options here](https://spark.apache.org/docs/latest/submitting-applications.html#master-urls)

In [2]:
# Create SparkSession
spark = SparkSession.builder\
             .master("local[1]")\
             .appName("spark-app-version-x")\
             .getOrCreate()

In [3]:
# Read taxi data
local_files = 'datasets/parquet/'
df = spark.read.parquet(local_files)

In [4]:
# DF is like a relation table in memory. Let's see the columns
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [5]:
# Query sample:
df.select('VendorID','total_amount', 'PULocationID').show(n=5)

+--------+------------+------------+
|VendorID|total_amount|PULocationID|
+--------+------------+------------+
|       2|        11.1|         238|
|       2|       76.49|         138|
|       1|       28.05|         140|
|       1|        24.7|         140|
|       2|       14.64|          79|
+--------+------------+------------+
only showing top 5 rows



In [6]:
# Query sample, using Spark SQL
df.createOrReplaceTempView('tbl_raw_yellow_taxis')

In [13]:
spark.sql('''
            select VendorID, tpep_pickup_datetime, passenger_count
            from tbl_raw_yellow_taxis
            where total_amount > 1 and passenger_count >2
        ''').show(n=5)

+--------+--------------------+---------------+
|VendorID|tpep_pickup_datetime|passenger_count|
+--------+--------------------+---------------+
|       2| 2023-03-01 00:22:22|              5|
|       2| 2023-03-01 00:33:12|              3|
|       2| 2023-03-01 00:11:32|              4|
|       2| 2023-03-01 00:08:01|              3|
|       2| 2023-03-01 00:02:37|              4|
+--------+--------------------+---------------+
only showing top 5 rows



In [7]:
# SQL Statement
# PULocationID = 188, 379 rows our of 3,066,766
spark.sql('''
          select min(tpep_pickup_datetime), max(tpep_dropoff_datetime)
          from tbl_raw_yellow_taxis
          ''').show(n=5)

+-------------------------+--------------------------+
|min(tpep_pickup_datetime)|max(tpep_dropoff_datetime)|
+-------------------------+--------------------------+
|      2001-01-01 00:06:49|       2023-05-03 23:19:31|
+-------------------------+--------------------------+



In [20]:
# SQL Statement
spark.sql('''
          select extract(year from tpep_pickup_datetime), count(1)
          from tbl_raw_yellow_taxis
          group by extract(year from tpep_pickup_datetime)
          order by count(1) desc
          ''').show(n=100)

+---------------------------------------+--------+
|extract(year FROM tpep_pickup_datetime)|count(1)|
+---------------------------------------+--------+
|                                   2023| 9605947|
|                                   2008|       9|
|                                   2001|       6|
|                                   2003|       4|
|                                   2002|       3|
|                                   2014|       1|
|                                   2009|       1|
+---------------------------------------+--------+



In [8]:
# SQL Statement
spark.sql('''
          select extract(year from tpep_pickup_datetime), count(1)
          from tbl_raw_yellow_taxis
          group by extract(year from tpep_pickup_datetime)
          having count(1) > 100
          ''').show(n=100)

+---------------------------------------+--------+
|extract(year FROM tpep_pickup_datetime)|count(1)|
+---------------------------------------+--------+
|                                   2023| 9605947|
+---------------------------------------+--------+



In [21]:
# SQL Statement example, using a subquery to clean the data
# Use case example: imagine our business users asked to us delete all data if dataset's year has < 100 rows.
df_clean_s1 = spark.sql('''
          select *
          from tbl_raw_yellow_taxis
          where extract(year from tpep_pickup_datetime) in
                        (select extract(year from tpep_pickup_datetime)
                        from tbl_raw_yellow_taxis
                        group by extract(year from tpep_pickup_datetime)
                        having count(1) > 100
                        )
          ''')

In [22]:
# Register new Temp View, using the cleansed new DataFrame 
df_clean_s1.createOrReplaceTempView('tbl_raw_yellow_taxis_clean_s1')

In [24]:
spark.sql('''
          select *
          from tbl_raw_yellow_taxis_clean_s1
          ''').show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2023-03-01 00:06:43|  2023-03-01 00:16:43|              1|          0.0|         1|                 N|         238|          42|           2|        8.6|  1.0|    0.5|       0.

In [25]:
# SQL Statement
spark.sql('''
          select min(tpep_pickup_datetime), max(tpep_dropoff_datetime)
          from tbl_raw_yellow_taxis_clean_s1
          ''').show(n=5)

+-------------------------+--------------------------+
|min(tpep_pickup_datetime)|max(tpep_dropoff_datetime)|
+-------------------------+--------------------------+
|      2023-01-31 23:49:00|       2023-05-03 23:19:31|
+-------------------------+--------------------------+



---
### If we want to write the output, for example partitioned by date

In [26]:
# Create new partition key
df_sink = df_clean_s1.withColumn("p_date",to_date(col('tpep_pickup_datetime')))

In [27]:
# Write to local storage, if not done already:
df_sink.write.partitionBy("p_date").mode("overwrite").parquet("datasets/yellow_taxis_daily/")

In [28]:
# Stop the session
spark.stop()